# Turning a Google Colab notebook into a web app

---

### This is the completed iris classifier notebook to be used as reference for Anvil's [turning a Colab notebook into a web app tutorial](https://anvil.works/learn/tutorials/google-colab-to-web-app).

The following code was added so you can connect this notebook to your Anvil app.

1. Added `getpass` prompt so you can add your apps Uplink key
2. Install the `anvil-uplink` library
3. Import the `anvil.server` package
4. Connect the notebook using your apps Uplink key
5. Create a function to call from your app that includes the `anvil.server.callable` decorator
6. Add `anvil.server.wait_forever()` to the end of the notebook

### See below for more detail.

---

### Enter the Uplink key from your Anvil web app.
For information on how to get your apps Uplink key, see [Step 4 - Enable the Uplink](https://anvil.works/learn/tutorials/google-colab-to-web-app#step-4-enable-the-uplink).

In [ ]:
from getpass import getpass
uplink_key = getpass('Enter your Uplink key: ')

Enter your Uplink key: ··········


### Install the `anvil-uplink` library.

In [ ]:
!pip install anvil-uplink

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 3.8 MB/s 
     |████████████████████████████████| 51 kB 244 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45229 sha256=fc0bcc9122ea007b56b413b0820b860c2ab774d77507e4a1cad6166059e9e564
  Stored in directory: /root/.cache/pip/wheels/29/ea/7d/3410aa0aa0e4402ead9a7a97ab2214804887e0f5c2b76f0c96
Successfully built ws4py


### Import the Anvil server package.

Importing `anvil.server` means, when this notebook is connected via the Uplink, it will behave in the same way as any other [Anvil Server Module](https://anvil.works/docs/server).

In [ ]:
import anvil.server

### Connect this notebook to your app via the Uplink.

In [ ]:
anvil.server.connect(uplink_key)

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


### Build and train the classification model

The next cell gets the dataset, finds an optimal number of neighbours and then builds and trains the model. How this works is outside the scope of this tutorial, however, if you want to read more about how the code below works, Towards Data Science has a useful article [here](https://towardsdatascience.com/knn-using-scikit-learn-c6bed765be75).


In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 , random_state=4)

# The following code is used only when needing to find the optimal n_neighbors
"""
scores = {}
scores_list = []
k_range = range(1, 26)
for k in k_range:
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(X_train, y_train)
  y_pred = knn.predict(X_test)
  scores[k] = metrics.accuracy_score(y_test, y_pred)
  scores_list.append(metrics.accuracy_score(y_test, y_pred))

plt.plot(k_range,scores_list)
plt.xlabel('Value of K for KNN')
plt.ylabel('Testing Accuracy')
"""

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X,y)

import joblib
from google.colab import files

model = joblib.dump(knn, "knn.skmodel")[0]
files.download(model)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Create our `predict_iris()` function with a `@anvil.server.callable` decorator. The decorator makes the function callable from our Anvil app.

In [ ]:
@anvil.server.callable
def predict_iris(sepal_length, sepal_width, petal_length, petal_width):
  classification = knn.predict([[sepal_length, sepal_width, petal_length, petal_width]])
  print(iris.target_names[classification][0])
  return iris.target_names[classification][0]

### Add `anvil.server.wait_forever()` function so the notebook is always available to the web app

In [ ]:
anvil.server.wait_forever()

KeyboardInterrupt: ignored

---

## That's it, 5 simple steps to connect your notebook to your Anvil app!

---

In [ ]:
!pip freeze requirements.txt